In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# ----------------------------
# 1) Chargement du dataset
# ----------------------------

# Assurez-vous que 'spam.csv' est dans le même dossier que ce script
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']

# Vérifier la distribution
print("Nombre de messages par classe :")
print(df['label'].value_counts())

# ----------------------------
# 2) Préprocessing du texte
# ----------------------------

def clean_text(text):
    text = text.lower()                      # mettre en minuscule
    text = re.sub(r'http\S+', '', text)      # supprimer URLs
    text = re.sub(r'\d+', '', text)          # supprimer chiffres
    text = re.sub(r'[^a-z\s]', '', text)     # garder lettres seulement
    text = re.sub(r'\s+', ' ', text).strip() # enlever espaces multiples
    return text

df['clean_message'] = df['message'].apply(clean_text)

# ----------------------------
# 3) Vectorisation
# ----------------------------

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['clean_message'])

# Encodage labels
y = df['label'].map({'ham': 0, 'spam': 1})

# ----------------------------
# 4) Séparation train/test
# ----------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# ----------------------------
# 5) Entraînement du modèle
# ----------------------------

model = MultinomialNB()
model.fit(X_train, y_train)

# ----------------------------
# 6) Évaluation
# ----------------------------

y_pred = model.predict(X_test)
print('Accuracy :', accuracy_score(y_test, y_pred))
print('Rapport de classification :\n', classification_report(y_test, y_pred))

# ----------------------------
# 7) Test sur un nouveau message
# ----------------------------

new_messages = [
    "Congratulations! You've won a free ticket to Bahamas. Click here.",
    "Hey, are we meeting for lunch today?",
    "Limited time offer: earn $1000 by working from home!"
]

new_clean = [clean_text(msg) for msg in new_messages]
new_vect = vectorizer.transform(new_clean)
predictions = model.predict(new_vect)

for msg, pred in zip(new_messages, predictions):
    print(f"Message : {msg}")
    print("Prediction :", "spam" if pred==1 else "ham")
    print('-'*50)


Nombre de messages par classe :
ham     4825
spam     747
Name: label, dtype: int64
Accuracy : 0.9706937799043063
Rapport de classification :
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      1448
           1       0.86      0.93      0.90       224

    accuracy                           0.97      1672
   macro avg       0.92      0.95      0.94      1672
weighted avg       0.97      0.97      0.97      1672

Message : Congratulations! You've won a free ticket to Bahamas. Click here.
Prediction : spam
--------------------------------------------------
Message : Hey, are we meeting for lunch today?
Prediction : ham
--------------------------------------------------
Message : Limited time offer: earn $1000 by working from home!
Prediction : ham
--------------------------------------------------
